In [1]:
import PIL
import subprocess
import re
import os
import time
import io
import cv2
import requests
import bs4
import jieba
import jieba.analyse
from IPython.display import display, HTML
import json

In [2]:
def capture_img():
    subprocess.call('.\\adb\\adb shell /system/bin/screencap -p /sdcard/screenshot.png')
    time.sleep(0.1)
    subprocess.call('.\\adb\\adb pull /sdcard/screenshot.png %s' % './tmp/screenshot.png')
    time.sleep(0.1)
    
    return cv2.imread('./tmp/screenshot.png')

In [3]:
%run key.py

In [4]:
from aip import AipOcr

client = AipOcr(AIP_APP_ID, AIP_API_KEY, AIP_SECRET_KEY)

In [5]:
def get_text_in_region(aip_rst, region):
    s = ''
    if 'words_result' in aip_rst:
        words = []
        for p in aip_rst['words_result']:
            loc = p['location']
            if loc['top'] > region[0] and loc['top'] + loc['height'] < region[1] \
                and loc['left'] > region[2] and loc['left'] + loc['width'] < region[3]:
                if loc['left'] < 1080 / 2: # only get words that starts from left
                    words.append(p['words'])
        s = ''.join(words)
        
    if s != '':
        # remove heading, like 1. A. B.
        if re.match(r"^A|B|C|\d+\.|,|。|，", s) != None: 
            ss = re.split(r"\.|,|。|，",s)
            if len(ss) > 1:
                s = ''.join(ss[1:])
            
    return s

In [6]:
def ocr(im, regions):
    _, byte_arr = cv2.imencode('.jpg', im, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
    aip_rst = client.general(byte_arr.tobytes())
    return [get_text_in_region(aip_rst, region) for region in regions]

In [7]:
def go_search(s, engine='google'):
    if engine == 'google':
        os.system('start www.google.com/search?q=%s' % s)
    elif engine == 'baidu':
        os.system('start www.baidu.com/s?wd=%s' % s)

In [8]:
def search(question):
    rst = requests.get('http://www.baidu.com/s?wd=%s&rn=50' % question)
    rst.encoding = 'utf-8'
    search_rst = rst.text
    
    soup = bs4.BeautifulSoup(search_rst, 'html.parser')
    rst = soup.select('div#content_left')[0]
    return rst.text

def search_advanced(question_words, opt_words, return_string=True, engine='baidu'):
    '''
    Advanced search
    question_words: list of words of the question
    opt_words: list of list of words of the options
    '''
    all_opt_words = []
    
    for words in opt_words:
        all_opt_words += words
        
    
    if engine == 'baidu':
        rst = requests.get('http://www.baidu.com/s?q1=%s&q2=&q3=%s&q4=&rn=50' % (
            '+'.join(question_words),
            '+'.join(all_opt_words)))
        print('http://www.baidu.com/s?q1=%s&q2=&q3=%s&q4=&rn=50' % (
            '+'.join(question_words),
            '+'.join(all_opt_words)))
        rst.encoding = 'utf-8'
        search_rst = rst.text

        soup = bs4.BeautifulSoup(search_rst, 'html.parser')
        rst = soup.select('div#content_left')[0]

        if return_string:
            return rst.text
        else:
            ret = []

            containers = rst.select('.c-container')

            for c in containers:
                datum = {}
                title = c.select('h3')
                title = title[0].text if len(title) > 0 else ''

                abstract = c.select('.c-abstract')
                abstract = abstract[0].text if len(abstract) > 0 else ''

                datum['title'] = title
                datum['abstract'] = abstract

                datum['html'] = str(c)
                ret.append(datum)
            return ret
            
    elif engine == 'google':
        'https://www.google.com.hk/search?num=50&source=hp&ei=G2dcWsidNJGajwPcurTgDA&q=a+bc&oq=a+bc&gs_l=psy-ab.3...1983.2905.0.3161.5.4.0.0.0.0.0.0..0.0....0...1c.1.64.psy-ab..5.0.0.0...0.ulvJkmLzPfA'
        'https://www.google.com.hk/search?hl=en&as_q=a+b&as_epq=&as_oq=c+d&as_eq=&as_nlo=&as_nhi=&lr=&cr=&as_qdr=all&as_sitesearch=&as_occt=any&safe=images&as_filetype=&as_rights='
    return '' if return_string else []

In [80]:
rst = requests.get('https://www.googleapis.com/customsearch/v1?key=%s&cx=%s&q=%s' % (
    G_KEY,
    G_CX,
    'hello world'), 
    proxies={'http':'127.0.0.1:1080','https':'127.0.0.1:1080'}, verify=False)

In [84]:
rst.text

'{\n "kind": "customsearch#search",\n "url": {\n  "type": "application/json",\n  "template": "https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&relatedSite={relatedSite?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json"\n },\n "queries": {\n  "request": [\n   {\n    "title": "Google Custom Search - hello world",\n    "totalResults": "65500000",\n    "searchTerms": "hello world",\n    "count": 10,\n    "startIndex": 1,\n    "inputEncoding

In [88]:
rst = json.loads(rst.text)
rst

{'context': {'title': 'Chiji'},
 'items': [{'cacheId': '_njBYFs_i0oJ',
   'displayLink': 'zh.wikipedia.org',
   'formattedUrl': 'https://zh.wikipedia.org/zh-hans/Hello_World',
   'htmlFormattedUrl': 'https://zh.wikipedia.org/zh-hans/<b>Hello</b>_<b>World</b>',
   'htmlSnippet': '<b>Hello</b>, <b>World</b>程序指的是指在计算机屏幕上输出“<b>Hello</b>, <b>World</b>!”（意为「你好，世界！」<br>\n）这行字符串的计算机程序。一般来说，这是每一种计算机编程语言中最基本、最简单<br>\n的程序，亦通常是初学者所编写的第一个程序。它还可以用来确定该语言的编译器、<br>\n程序开发环境，以及运行环境是否已经安装妥當。 将输出字符串“<b>Hello World</b>”作为第<br>\n&nbsp;...',
   'htmlTitle': '<b>Hello World</b> - 维基百科，自由的百科全书',
   'kind': 'customsearch#result',
   'link': 'https://zh.wikipedia.org/zh-hans/Hello_World',
   'pagemap': {'cse_image': [{'src': 'https://upload.wikimedia.org/wikipedia/commons/a/ae/Hello_World%21_in_AutoIt%21_3.2_on_Windows_Vista.png'}],
    'cse_thumbnail': [{'height': '181',
      'src': 'https://encrypted-tbn2.gstatic.com/images?q=tbn:ANd9GcT6myvXx9yR0-HmOspu7mcgzg3TxK8VKuRekVYOCYe9hoC2eMaJJXx32M7U',
      'width':

In [9]:
def occur_count(s,words):
    n = 0
    for word in words:
        n += len(re.findall(word, s))
    return n


def score(search_rst, opts_words_search):
    scores = [0 for i in opts_words_search]
    decay = 0.5
    weight = 1
    
    for c in search_rst:
        title = c['title']
        abstract = c['abstract']
        
        b_find_any = False
        for i, opt_words in enumerate(opts_words_search):
            for w in opt_words:
                if w in title:
                    scores[i] += weight
                    b_find_any = True
                    break
                elif w in abstract:
                    scores[i] += weight * 0.5
                    b_find_any = True
                    break
    
        if b_find_any:
            weight *= decay
        
    return scores

In [10]:
def keywords(question, opt_a, opt_b, opt_c):
    quest_words = jieba.analyse.extract_tags(question, allowPOS=['n','nr','ns','nt','nz','nl','ng'])
    opt_words = jieba.analyse.extract_tags(' '.join([opt_a,opt_b,opt_c]), allowPOS=['n','nr','ns','nt','nz','nl','ng'])
    words = quest_words + opt_words
    return words

In [11]:
def dump_question(question, opt_a, opt_b, opt_c):
    with open('questions.txt','a+') as f:
        f.write(','.join([question, opt_a, opt_b, opt_c]) + '\n')

In [18]:
def search_ans(question, opt_a, opt_b, opt_c):

    # question might be too long
    len_q = len(question)
    len_opt = len(opt_a) + len(opt_b) + len(opt_c)

    if len_q + len_opt > 37:
        words = jieba.analyse.extract_tags(question)
        quest_words = []
        l = 0

        for w in reversed(words):
            l += len(w)
            if l < 37 - len_opt:
                quest_words.append(w)
            else:
                break

    else:
        quest_words = [question]

    opts = [opt_a, opt_b, opt_c]
    opt_words = []

    for opt in opts:
#             opt_words.append(list(map(lambda p:p.word, filter(lambda p:p.flag.startswith('n'), 
#                                                  jieba.posseg.cut(opt)))))
        if len(opt) > 4:
            opt_words.append(jieba.analyse.extract_tags(opt))
        else:
            opt_words.append([opt])

#         rst = search(question)
#         time.sleep(0.1)
#         rst += search('%20'.join(quest_words))
    search_rst = search_advanced(quest_words, opt_words, return_string=False)
#         print(rst)


    opt_w_search = [jieba.lcut_for_search(o) for o in opts]

    # 选项间去重, 仍然有bug：如:"A. 金，B. 金刚石"的场景
    common_words = set(opt_w_search[0]).intersection(set(opt_w_search[1])).intersection(set(opt_w_search[2]))

    if len(common_words) != 0:
        opt_w_search_reduced = [list(filter(lambda word:not word in common_words, opt_w)) for opt_w in opt_w_search]
        opt_w_search_reduced = [opt_w_search_reduced[i] if len(opt_w_search_reduced[i]) > 0 else opt_w_search[i]\
                               for i in range(3)]
    else:
        opt_w_search_reduced = opt_w_search
#     print(opt_w_search_reduced)


    scores = score(search_rst, opt_w_search_reduced)
    # 考虑反向问题，比方说在战国初期下列哪一个国家*未*与燕国相邻?
    neg_words = ['不','未','没有']

    neg_question = False
    for nw in neg_words:
        if nw in question:
            neg_question = True
#             print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
#             print('!!               疑似反向问题              !!!')
#             print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            display(HTML('<h2>疑似反向问题!!</h2>'))
            break

    ans = [('A',opt_a,scores[0]),
          ('B',opt_b,scores[1]),
          ('C',opt_c,scores[2])]

    print('question:', question)
    for p in sorted(ans, key=lambda p:p[2], reverse=True):
        print(p)

    # Show top search result
#     print('----------------------------')
    for i in range(3):
#         print(rst_soup.select('.c-container')[i])
        display(HTML(search_rst[i]['html']))
#     print('----------------------------')

#         go_search(question)

    dump_question(question, opt_a, opt_b, opt_c)

In [13]:
def search_ans_from_pic(im, game='baiwan'):
    print('Parsing pic...')
    
    if game == 'baiwan':
        #百万英雄
        question, opt_a, opt_b, opt_c = ocr(im,
                       [(300,600,0,1080),
                       (655,815,0,1080),
                       (845,1005,0,1080),
                       (1035,1200,0,1080)])
    elif game == 'zhishi':
        #芝士超人
        question, opt_a, opt_b, opt_c = ocr(im,
                       [(280,580,0,1080),
                       (580,735,0,1080),
                       (775,930,0,1080),
                       (970,1125,0,1080)])
    elif game == 'chongding':
        #冲顶大会
        question, opt_a, opt_b, opt_c = ocr(im,
                       [(305,645,0,1080),
                       (645,775,0,1080),
                       (805,940,0,1080),
                       (970,1105,0,1080)])
    else:
        return

    if question != '':
        search_ans(question, opt_a, opt_b, opt_c)

    else:
        print('No question found!')

In [15]:
im = cv2.imread('./tmp/screenshot.png')
im = im[:1200,:,:]

In [16]:
search_ans_from_pic(im, game='baiwan')

Parsing pic...
http://www.baidu.com/s?q1=电影《中国合伙人》中由主角创办的英语学校叫什么名字?&q2=&q3=新希望+新东方+新梦想&q4=&rn=50
question: 电影《中国合伙人》中由主角创办的英语学校叫什么名字?
('A', '新希望', 1.9999999999999964)
('C', '新梦想', 1.9999999999999964)
('B', '新东方', 1.9606933593731775)


In [19]:
while True:
    ipt = input()
    if ipt == 'q':
        break
    else:
        im = capture_img()
        search_ans_from_pic(im[:1200,:,:], game='baiwan')


Parsing pic...
http://www.baidu.com/s?q1=在《三国演义》中下列人物谁最先拥有赤兔马?&q2=&q3=吕布+曹操+董卓&q4=&rn=50
question: 在《三国演义》中下列人物谁最先拥有赤兔马?
('B', '曹操', 1.1539082685490598)
('C', '董卓', 0.9953461773646808)
('A', '吕布', 0.4346822626197593)



Parsing pic...
http://www.baidu.com/s?q1=一般情况下水能和以下哪种物质互溶?&q2=&q3=大豆油+甘油+玉米油&q4=&rn=50
question: 一般情况下水能和以下哪种物质互溶?
('A', '大豆油', 1.3273778557814921)
('B', '甘油', 0.02415418438232564)
('C', '玉米油', 6.224215030670166e-05)



Parsing pic...
http://www.baidu.com/s?q1=在我国古代称呼中下列哪一个称呼代指的年龄最小?&q2=&q3=舞象之年+舞勺之年+弱冠&q4=&rn=50
question: 在我国古代称呼中下列哪一个称呼代指的年龄最小?
('B', '舞勺之年', 1.9999999999999982)
('A', '舞象之年', 1.9843749999999982)
('C', '弱冠', 0.2519533658050932)



Parsing pic...
http://www.baidu.com/s?q1=40摄氏度与100华氏度哪个温度更高?&q2=&q3=40+摄氏度+华氏度+100+样高&q4=&rn=50
question: 40摄氏度与100华氏度哪个温度更高?
('A', '40摄氏度', 1.990234375)
('B', '100华氏度', 1.44921875)
('C', '样高', 0.001953125)


摄氏度和华氏度什么意思?,2个回答,2015-03-22
摄氏度和华氏度哪个更精确,2个回答,2009-10-11
102.8华氏度等于多少摄氏度,2个回答,2016-02-16


...


Parsing pic...
http://www.baidu.com/s?q1=以下+作者+人生+自是&q2=&q3=声声慢+寻寻觅觅+生查子+元夕+纤云弄巧+鹊桥&q4=&rn=50


question: “人生自是有情痴此恨不关风与月”的作者还写过以下哪部词作?
('B', '《生查子元夕》', 1.0332040935754776)
('C', '《鹊桥仙·纤云弄巧》', 0.8720669597387314)
('A', '《声声慢·寻寻觅觅》', 0.8225145488977432)


李清照《声声慢》,7个回答,2009-08-30
声声慢 李清照,2个回答,2015-07-27


厦门大学网络教育2015-2016学年第一学期大学....doc,评分:0/5,3页


q


In [14]:
with open('questions.txt','r') as f:
    i = 0
    for line in f:
        question, opt_a, opt_b, opt_c = map(lambda s:s.strip(), line.split(','))
        search_ans(question, opt_a, opt_b, opt_c)
        time.sleep(1)
        i += 1
        if i > 5:
            break

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wangz\AppData\Local\Temp\jieba.cache
Loading model cost 0.811 seconds.
Prefix dict has been built succesfully.


http://www.baidu.com/s?q1=以下哪个海沟最深?&q2=&q3=马里亚纳+海沟+波多黎各+海沟+海沟+菲律宾&q4=&rn=50
question: 以下哪个海沟最深?
('A', '马里亚纳海沟', 1.4298410713672638)
('C', '菲律宾海沟', 1.046264659613371)
('B', '波多黎各海沟', 0.0068359375)


世界上最深的海沟位于哪个大洋?,2个回答,2016-11-27
有人知道马尼亚拉海沟么?,2个回答,2011-08-05


http://www.baidu.com/s?q1=“桃李满天下”中的“桃李”一般用来指代?&q2=&q3=老师+学生+前辈&q4=&rn=50
question: “桃李满天下”中的“桃李”一般用来指代?
('A', '老师', 1.9120785841323595)
('B', '学生', 0.10356269609258106)
('C', '前辈', 0.004882849814748624)


形容孩子对老师爱的成语,1个回答,2016-10-26
古文中还可以把“弟子”(学生)称做什么?(用于...,2个回答,2015-09-09
老师能用什么词来形容,5个回答,2007-08-17


...

http://www.baidu.com/s?q1=《马里奥》系列游戏中马里奥吃哪种植物可以体积变大?&q2=&q3=蘑菇+土豆+西红柿&q4=&rn=50
question: 《马里奥》系列游戏中马里奥吃哪种植物可以体积变大?
('A', '蘑菇', 0.765625)
('C', '西红柿', 0.3515625)
('B', '土豆', 0)


http://www.baidu.com/s?q1=在战国初期下列哪一个国家未与燕国相邻?&q2=&q3=赵国+楚国+齐国&q4=&rn=50


question: 在战国初期下列哪一个国家未与燕国相邻?
('A', '赵国', 1.7529320729663596)
('C', '齐国', 0.9999999999417923)
('B', '楚国', 0.00830936839338392)


我收集的赵国资料..._赵国吧,,


http://www.baidu.com/s?q1=吴承恩所著的《西游记》中牛魔王会多少种变化?&q2=&q3=36种+64种+72种&q4=&rn=50
question: 吴承恩所著的《西游记》中牛魔王会多少种变化?
('A', '36种', 1.149935007095337)
('C', '72种', 0.31249999161809683)
('B', '64种', 0.1328125149011612)
